In [ ]:
# EEG Deep Learning Workflow for Real-Time Smart Home Control using EEGET-ALS Dataset (EDF version, PhysioIntent commented out)

import os
import glob
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import mne  # <-- EDF reader

# load PhysioIntent Dataset
# def load_physiointent_data(path="./PhysioIntent"):
#     pass


#load EEGET - ALS Dataset
def load_eeget_als_data(path="/Users/kylegarcia/Desktop/EEGET-ALS Dataset"):
    """Loads EEG data from EEGET-ALS dataset (.edf files)."""
    print("🧠 Scanning for EEG .edf files (deep recursive search)...")

    edf_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.lower().endswith(".edf"):
                full_path = os.path.join(root, file)
                edf_files.append(full_path)

    if not edf_files:
        raise RuntimeError(f"No EDF files found under {os.path.abspath(path)}")

    print(f"✅ Found {len(edf_files)} EDF files")
    print("📁 Example files:")
    for f in edf_files[:5]:
        print(f"   {f}")

    # List of EEG channels to extract
    target_channels = ["Fp1", "Fp2", "TP7", "TP8"]

    all_data = []
    all_labels = []

    for f in edf_files:
        try:
            raw = mne.io.read_raw_edf(f, preload=True, verbose=False)
            available_channels = raw.ch_names
            selected = [ch for ch in target_channels if ch in available_channels]

            if not selected:
                print(f"⚠️ Skipping {f} (no matching EEG channels)")
                continue

            data, _ = raw.get_data(picks=selected, return_times=True)

            # Transpose: samples x channels
            data = data.T

            # Create placeholder label (you can replace later)
            label = 0  # For now all data labeled 0; can modify later for class labeling

            all_data.append(data)
            all_labels.extend([label] * len(data))

            print(f"📄 Loaded {f} | shape: {data.shape}, channels: {selected}")

        except Exception as e:
            print(f"⚠️ Error loading {f}: {e}")

    if not all_data:
        raise RuntimeError("No valid EDF files loaded after scanning.")

    X = np.vstack(all_data)
    y = np.array(all_labels)

    print(f"✅ Combined EEG dataset shape: {X.shape}")
    print(f"🧩 Labels: {np.unique(y)}")

    return X, y


# dataset preparation (Normalization + Train/Val Split)

def prepare_combined_dataset():
    """Loads EEGET-ALS EDF dataset and normalizes."""
    X2, y2 = load_eeget_als_data()

    # Normalize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X2)

    # Split
    X_train, X_val, y_train, y_val = train_test_split(
        X_scaled, y2, test_size=0.2, random_state=42, stratify=y2 if len(np.unique(y2)) > 1 else None
    )

    print(f"✅ Dataset ready — Train: {X_train.shape}, Val: {X_val.shape}")
    return (X_train, y_train), (X_val, y_val)


# simple EEG Classifier (Multi-Layer Perceptron)

def train_model(train_ds, val_ds):
    """Train a simple MLP classifier."""
    X_train, y_train = train_ds
    X_val, y_val = val_ds

    print("🚀 Training MLPClassifier on EEG data...")
    model = MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation="relu",
        solver="adam",
        max_iter=100,
        random_state=42,
        verbose=True
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    print("\n📊 Validation Report:\n", classification_report(y_val, preds, zero_division=0))
    return model


# Real-Time EEG Smart Home Control (Simulated Input)

def realtime_eeg_control(model):
    """Simulates live EEG data control loop."""
    print("\n--- Realtime EEG Smart Home Control ---\n")
    try:
        while True:
            # Simulate live EEG readings (5 channels)
            eeg_input = np.random.randn(1, 5)
            pred = model.predict(eeg_input)[0]
            print(f"→ Predicted Action: {pred}")
            time.sleep(1.0)
    except KeyboardInterrupt:
        print("🛑 Stopped real-time loop.")


# main
if __name__ == "__main__":
    train_ds, val_ds = prepare_combined_dataset()
    model = train_model(train_ds, val_ds)
    realtime_eeg_control(model)
